In [4]:
import json
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer,GenerationConfig
torch.set_default_dtype(torch.float16) 
import os

In [5]:
def save_list_to_json(lst, filename):
    """Save Files

    """
    
    with open(filename, 'w') as file:
        json.dump(lst, file)

In [6]:
def rm_file(file_path):
    """Delete Files

    """
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"File {file_path} removed successfully.")


In [7]:
model_name = "microsoft/DialoGPT-large"
save_file = f'qa_output/gpt.json'

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          device_map="auto")

prefix = "Below is a question followed by some context from different sources. Please answer the question based on the context. The answer to the question is a word or entity. If the provided information is insufficient to answer the question, respond 'Insufficient Information'. Answer directly without explanation."


/home/anishkav/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
# toy_data/voyage-02_rerank_retrieval_test.json is a file saved the last step (retrieve).
with open('step1_data.json', 'r') as file:
    doc_data = json.load(file)


In [9]:
def query_bot(
        messages,
        temperature=0.1,
        max_new_tokens=512,
        **kwargs,
):
    # Tokenize the input message and get input_ids and attention_mask
    max_length = 512
    inputs = tokenizer(
        messages,
        max_length=max_length,      # Change this to the maximum you want (e.g., 256, 512)
        truncation=True,     # Truncate longer sequences
        return_tensors='pt'
    )
    max_length = 512
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask  # Added attention mask
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = input_ids[:, :max_length]
    attention_mask = attention_mask[:, :max_length]
    model.to(device)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)


    # Set generation configuration
    generation_config = GenerationConfig(
        temperature=temperature,
        do_sample=True,  # Explicitly set do_sample to enable sampling
        **kwargs,
    )

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,  # Pass attention mask here
            generation_config=generation_config,
            pad_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )

    s = generation_output.sequences[0]
    output = tokenizer.decode(s, skip_special_tokens=True)
    return output


In [ ]:
rm_file(save_file)
save_list = []
for d in tqdm(doc_data):
    retrieval_list = d['retrieval_list']
    context = '--------------'.join(e['text'] for e in retrieval_list)
    prompt = f"{prefix}\n\nQuestion:{d['query']}\n\nContext:\n\n{context}"
    response = query_bot(prompt)
    print(response)
    save = {}
    save['query'] = d['query']
    save['prompt'] = prompt
    save['model_answer'] = response
    save['gold_answer'] = d['answer']
    save['question_type'] = d['question_type']
    save_list.append(save)

save_list_to_json(save,save_file)

File qa_output/gpt.json removed successfully.


  0%|          | 0/1194 [00:00<?, ?it/s]

  1%|          | 6/1194 [00:00<01:04, 18.38it/s]

Below is a question followed by some context from different sources. Please answer the question based on the context. The answer to the question is a word or entity. If the provided information is insufficient to answer the question, respond 'Insufficient Information'. Answer directly without explanation.

Question:Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?

Context:

[Excerpt from document]
title: The FTX trial is bigger than Sam Bankman-Fried
published_at: 2023-09-28T12:00:00+00:00
source: The Verge
Excerpt:
-----
“This trial is going to be an excruciating moment for the industry because no one knows what kind of evidence might come out.”

Bankman-Fried, the founder of FTX and Alameda Research, is facing seven counts of criminal charges: two counts of wire fraud, and five counts of conspiracy

  1%|▏         | 16/1194 [00:00<00:34, 33.81it/s]

...

Below is a question followed by some context from different sources. Please answer the question based on the context. The answer to the question is a word or entity. If the provided information is insufficient to answer the question, respond 'Insufficient Information'. Answer directly without explanation.

Question:Does 'The Guardian' report on 'Game 3 of the World Series' indicate that 'Brandon Pfaadt' is a probable starter, while 'Sporting News' suggests his pitching performance ended at a specific event during the game?

Context:

[Excerpt from document]
title: World Series Game 2: Arizona Diamondbacks v Texas Rangers – live
published_at: 2023-10-28T23:00:28+00:00
source: The Guardian
Excerpt:
-----
What happens next is anybody’s guess. The World Series heads to Arizona for Monday’s Game 3 where Rangers’ Max Scherzer and the Diamondbacks’ Brandon Pfaadt are currently penciled in as the probable starters. Stay tuned to the Guardian for further coverage of the 2023 World Series as it 

 99%|█████████▉| 1186/1194 [00:34<00:00, 40.71it/s]

Below is a question followed by some context from different sources. Please answer the question based on the context. The answer to the question is a word or entity. If the provided information is insufficient to answer the question, respond 'Insufficient Information'. Answer directly without explanation.

Question:Does the TechCrunch article discussing the launch of DeepMind's next-generation chatbot Gemini suggest a different development in AI chatbot technology compared to the TechCrunch article about the general-purpose AI chatbot ChatGPT?

Context:

[Excerpt from document]
title: One year later, ChatGPT is still alive and kicking
published_at: 2023-11-30T14:10:43+00:00
source: TechCrunch
Excerpt:
-----
Google scrambled to launch a response to ChatGPT, eventually releasing Bard, a more or less comparable AI chatbot, in February. Countless other ChatGPT rivals and derivatives have arrived to market since, most recently Amazon Q, a more business-oriented take on ChatGPT. DeepMind, Go

100%|██████████| 1194/1194 [00:34<00:00, 34.16it/s]

Below is a question followed by some context from different sources. Please answer the question based on the context. The answer to the question is a word or entity. If the provided information is insufficient to answer the question, respond 'Insufficient Information'. Answer directly without explanation.

Question:Which company, featured in articles by TechCrunch, The Verge, and Fortune, had significant holdings in "unlocked FTT" and "FTT collateral," was able to maintain a negative balance without liquidation on FTX, and was involved in taking losses to present a better financial image for FTX, all while acting within its legal boundaries as a customer, payment processor, and market maker?

Context:

[Excerpt from document]
title: Today the FTX jury suffered through a code review
published_at: 2023-10-06T22:21:46+00:00
source: The Verge
Excerpt:
-----
Other customers didn’t have the same privileges.

Alameda also allegedly kept FTX from looking bad by assuming some of its losses. In 